In [1]:
library(Seurat)
library(openxlsx)
library(readxl)
library(Matrix)
library(readr)

Loading required package: SeuratObject

Loading required package: sp

‘SeuratObject’ was built under R 4.3.2 but the current version is
4.3.3; it is recomended that you reinstall ‘SeuratObject’ as the ABI
for R may have changed

‘SeuratObject’ was built with package ‘Matrix’ 1.6.3 but the current
version is 1.6.5; it is recomended that you reinstall ‘SeuratObject’ as
the ABI for ‘Matrix’ may have changed


Attaching package: ‘SeuratObject’


The following object is masked from ‘package:base’:

    intersect




In [8]:
#Need to find out if all duplicates are nonzeros
#Rama
setwd("/dh-projects/ag-conrad/analysis/saal11/panfib/Renaming/Rama")
#Upload matrices
data_rama_matrix <- readMM("data_rama_matrix.mtx")
#Upload original gene list
data_rama_genes <- read.delim("data_rama_genes.tsv")
data_rama_genes <- data_rama_genes$genes
data_rama_cellIDs <- read.delim("data_rama_cellIDs.tsv")
rownames(data_rama_matrix) <- data_rama_genes
colnames(data_rama_matrix) <- data_rama_cellIDs$cellIDs

In [11]:
#Upload genes of interest (duplicates)
data_rama_genes_dup <- read.xlsx("data_rama_geneIDs_duplicates_forcount.xlsx")

In [14]:
#Find the zeros of these duplicates
#Initialize an empty list to store the sums
sums_list <- list()

#Iterate over each gene in your list
for (gene in data_rama_genes_dup$Old) {
  #Subset the count matrix for the current gene and calculate the sum
  gene_sum <- sum(data_rama_matrix[rownames(data_rama_matrix) == gene, ])
  
  #Add the sum to the sums_list with the gene name as the key
  sums_list[[gene]] <- gene_sum
}

In [ ]:
sums_list

In [18]:
#Upload manually edited geneIDs
data_rama_genes_manual <- read.xlsx("data_rama_geneIDs_int_removed_manualedit.xlsx")
rownames(data_rama_matrix) <- data_rama_genes_manual$New

In [19]:
#Check duplicates
#Find sums of duplicate genes (from manual edit)
duplicate_genes <- duplicated(rownames(data_rama_matrix))
cat("Duplicate gene IDs:\n")
print(rownames(data_rama_matrix)[duplicate_genes])
duplicated <- rownames(data_rama_matrix)[duplicate_genes]

Duplicate gene IDs:
 [1] "BTBD8"        "HNRNPU"       "TRAF3IP2-AS1" "S1PR3"        "LCOR"        
 [6] "MYO18A"       "SOCS4"        "IGFLR1"       "ROPN1L"       "ARHGAP42"    
[11] "MIA2"         "MEG8"         "GCNT4"        "RRM2"         "MICAL2"      
[16] "TMEM8B"       "LINC00632"    "NEBL"         "MEG8"         "ZFHX3"       
[21] "TBC1D3B"      "DLGAP2"       "UBE2R2-AS1"   "CC2D2B"       "SIGLEC12"    


In [ ]:
#Matches the decision list

In [21]:
#Filter zeros
row_sums <- rowSums(data_rama_matrix)
names_with_zeros <- names(row_sums[row_sums == 0])
length(names_with_zeros)

[1] 456

In [23]:
filtered_data_rama_matrix <- data_rama_matrix[row_sums != 0, ]

In [24]:
str(data_rama_matrix)
str(filtered_data_rama_matrix)

Formal class 'dgTMatrix' [package "Matrix"] with 6 slots
  ..@ i       : int [1:81916773] 78 99 103 183 188 198 222 224 225 226 ...
  ..@ j       : int [1:81916773] 0 0 0 0 0 0 0 0 0 0 ...
  ..@ Dim     : int [1:2] 23498 58358
  ..@ Dimnames:List of 2
  .. ..$ : chr [1:23498] "FO538757.2" "AP006222.2" "RP4-669L17.10" "RP11-206L10.9" ...
  .. ..$ : chr [1:58358] "Healthy1_Cd45+_AAACCTGCAGTATCTG" "Healthy1_Cd45+_AACTGGTTCATGGTCA" "Healthy1_Cd45+_AACTTTCCAGCTGTAT" "Healthy1_Cd45+_AAGGTTCCACATGACT" ...
  ..@ x       : num [1:81916773] 1 1 1 1 1 1 1 9 4 11 ...
  ..@ factors : list()
Formal class 'dgTMatrix' [package "Matrix"] with 6 slots
  ..@ i       : int [1:81916773] 78 99 103 183 188 198 222 224 225 226 ...
  ..@ j       : int [1:81916773] 0 0 0 0 0 0 0 0 0 0 ...
  ..@ Dim     : int [1:2] 23042 58358
  ..@ Dimnames:List of 2
  .. ..$ : chr [1:23042] "FO538757.2" "AP006222.2" "RP4-669L17.10" "RP11-206L10.9" ...
  .. ..$ : chr [1:58358] "Healthy1_Cd45+_AAACCTGCAGTATCTG" "Healthy1_Cd45+_A

In [25]:
#Concatenating replicates
filtered_data_rama_matrix <- as(filtered_data_rama_matrix, "dgCMatrix")

'as(<dgTMatrix>, "dgCMatrix")' is deprecated.
Use 'as(., "CsparseMatrix")' instead.
See help("Deprecated") and help("Matrix-deprecated").



In [26]:
gene_names <- rownames(filtered_data_rama_matrix)
unique_gene_names <- unique(gene_names)
gene_factors <- factor(gene_names, levels = unique_gene_names)

In [27]:
aggregated_counts <- rowsum(as.matrix(filtered_data_rama_matrix), group = gene_factors)

Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 10.0 GiB”


In [28]:
str(aggregated_counts)
#Number matches with the decision table

 num [1:23019, 1:58358] 0 0 0 0 0 0 0 0 0 0 ...
 - attr(*, "dimnames")=List of 2
  ..$ : chr [1:23019] "FO538757.2" "AP006222.2" "RP4-669L17.10" "RP11-206L10.9" ...
  ..$ : chr [1:58358] "Healthy1_Cd45+_AAACCTGCAGTATCTG" "Healthy1_Cd45+_AACTGGTTCATGGTCA" "Healthy1_Cd45+_AACTTTCCAGCTGTAT" "Healthy1_Cd45+_AAGGTTCCACATGACT" ...


In [29]:
#Creating a seurat object
data_rama_metadata <- read_csv("data_rama_metadata_final.csv")

New names:
• `` -> `...1`
• `...1` -> `...2`
Rows: 58358 Columns: 18
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (12): ...2, condition, aetiology, cells, lineage, subclass, study, tissu...
dbl  (5): ...1, nCount_RNA, nFeature_RNA, percent.mt, age
lgl  (1): smoking

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [30]:
data_rama_metadata <- as.data.frame(data_rama_metadata)

In [32]:
rownames(data_rama_metadata) <- data_rama_metadata$`...2`

In [34]:
data_rama_metadata$`...2` <- NULL
data_rama_metadata$`...1` <- NULL

In [36]:
data_rama_seurat <- CreateSeuratObject(counts = aggregated_counts, meta.data = data_rama_metadata)

Warning message:
“Data is of class matrix. Coercing to dgCMatrix.”


In [39]:
save(data_rama_seurat, file = "data_rama_seurat_final.RData")